In [1]:
!pip install tqdm --user

In [2]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
import json


In [3]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
import json


In [5]:
!pip install tqd

ERROR: Could not find a version that satisfies the requirement tqd (from versions: none)
ERROR: No matching distribution found for tqd


In [6]:
!pip install -e git+https://github.com/tqdm/tqdm.git@master#egg=tqdm

Obtaining tqdm from git+https://github.com/tqdm/tqdm.git@master#egg=tqdm
  Updating c:\makine_ogrenimi_ve_uygulamalari\let1\src\tqdm clone (to revision master)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for tqdm (pyproject.toml): started
  Building editable for tqdm (pyproject.toml): finished with status 'done'
  Created wheel for tqdm: filename=tqdm-4.66.1-0.editable-py3-none-any.whl size=22420 sha256=19e88399ff07207a871fb4809b026f4ecb99f8812a3fa40b81215e0926b982dd
  Stored in directory: C:\Users\ozkan\AppData\Local\Temp

  Running command git fetch -q --tags
  Running command git reset --hard -q 4c956c20b83be4312460fc0c4812eeb3fef5e7df


In [29]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd

stop_words = set(stopwords.words('english'))

In [30]:
class Drug:
    """
    docstring for Drug.
    btk akedemi algortima analizi 
    """
    def __init__(self, features):

        self.id = features['id']
        self.name = features['name']
        self.state = features['state']
        self.kingdom = features['kgd']
        self.superclass = features['sclass']
        self.interaction = features['itrc']
        self.pathways = features['pathways']
        self.toxicity = features['toxicity']
        self.food_interactions = features['food_interactions']
        self.target = []

    def getDrugfeatures(self):
        drug_dict = {"drug_id":self.id,
                    "drug_name":self.name,
                    "drug_state":self.state,
                    "drug_kingdom":self.kingdom,
                    "drug_superclass":self.superclass,
                    "drug_interactions":self.interaction,
                    "drug_pathways":self.pathways,
                    "drug_toxicity":self.toxicity,
                    "drug_food_interactions":self.food_interactions}
        return drug_dict

    def addTarget(self, feature_target):
        self.target.append(feature_target)

In [31]:
dB_file = 'full_database.xml'
organism = 'Humans'
saveFile = 'drugBank_v515.json'

In [4]:

def summarize_sentences(sentencesV):
    
    doc = nlp(sentencesV)
    summeryText = ''
    summary_sentences = doc._.textrank.summary(limit_phrases=2, limit_sentences=1)
    return ' '.join([sent.text for sent in summary_sentences])

In [5]:
xtree = ET.parse(dB_file)
xroot = xtree.getroot()
drugs = list(xroot)


In [28]:
drug_targets = []

for i in tqdm(range(len(drugs))):
    drug = drugs[i]
    idDB = drug[0].text # Drug Bank ID
    

    for idx,feature in enumerate(drug):
        if 'name' in str(feature): # drug name
            drug_name = drug[idx].text
        
        if 'state' in str(feature):
            drug_state = drug[idx].text

        if 'classification' in str(feature): #type of drug   
            drug_class_kingdom = list(drug[idx])[2].text
            drug_class_superclass = list(drug[idx])[3].text
            

        if 'drug-interactions' in str(feature): #interaction other drugs
            
            drug_interaction_temp = '$'.join([di[2].text for di in list(drug[idx])])
            sentence_temp = drug_interaction_temp.split('$')
            drug_interaction =  [' '.join([word for word in word_tokenize(sentence) if not word in stop_words]) for sentence in sentence_temp]

           

        if 'pathways' in str(feature): #related pathways
            
            drug_pathway = [pathway[1].text \
                                    for pathway in list(drug[idx])]
        

        if 'targets' in str(feature): #if polypeptide, drug's targets
            targets = list(drug[idx])
            
        
        if 'toxicity' in str(feature):
            
                
            drug_toxicity_temp = f"${drug[idx].text}"
            sentences = drug_toxicity_temp.split('$')
            drug_toxicity =  [' '.join([word for word in word_tokenize(sentence) if not word in stop_words]) for sentence in sentences]
            
            
                
        if 'food-interactions' in str(feature):
            
            drug_food_interactions_temp = '$'.join([di.text for di in list(drug[idx])])
            sentences = drug_food_interactions_temp.split('$')
            drug_food_interactions =  [' '.join([word for word in word_tokenize(sentence) if not word in stop_words]) for sentence in sentences]
            
        
            

    # get all drug-related information in a dictionary
    if [""] in [drug_name, drug_class_kingdom, drug_class_superclass,drug_interaction,drug_pathway,drug_toxicity,drug_food_interactions]:
        continue 
    
    drug_dict = {"id":idDB,
                "name":drug_name,
                 "state":drug_state,
                "kgd":drug_class_kingdom,
                "sclass":drug_class_superclass,
                "itrc":drug_interaction,
                "pathways":drug_pathway,
                "toxicity":drug_toxicity,
                "food_interactions":drug_food_interactions}
    drug = Drug(drug_dict)

    # get information of polypeptide targets
    if len(targets) > 0:
        for target in targets:
            idx_pep = None
            # get indexes
            for idx,feature in enumerate(target): # check features of targets
                if 'organism' in str(feature):
                    idx_org = idx
                if 'name' in str(feature):
                    idx_name = idx
                if 'actions' in str(feature):
                    idx_act = idx
                if 'polypeptide' in str(feature):
                    idx_pep = idx

            # Get information for polypeptide
            if target[idx_org].text == organism:

                target_name = target[idx_name].text

                actions = ';'.join([action.text
                                    for action in list(target[idx_act])])

                # Get information for polypeptide
                if idx_pep is not None: #if there is polypeptide's info...
                    for idx,feature in enumerate(target[idx_pep]):
                        if 'gene-name' in str(feature):
                            gene_name = target[idx_pep][idx].text
                        if 'cellular-location' in str(feature):
                            cell_loc = target[idx_pep][idx].text
                        if 'external-identifiers' in str(feature):
                            for ext_id in list(target[idx_pep][idx]):
                                if ext_id[0].text == "UniProtKB":
                                    uniprot = ext_id[1].text
                else:
                    gene_name = None
                    action = None
                    cell_loc = None
                    uniprot = None

                row = {
                        "drug_id":drug.id,
                        "drug_name":drug.name,
                        "drug_state":drug.state,
                        "drug_kingdom":drug.kingdom,
                        "drug_superclass":drug.superclass,
                        "drug_interactions":drug.interaction,
                        "drug_pathways":drug.pathways,
                        "drug_toxicity":drug.toxicity,
                        "drug_food-interactions":drug_food_interactions,
                        "target_name":target_name,
                        "target_uniprot":uniprot,
                        "target_gene_name":gene_name,
                        "action":actions,
                        "cell_loc":cell_loc}

                drug_targets.append(row)
                

  0%|                                                                                | 1/15235 [00:00<19:26, 13.06it/s]


In [27]:
print(drug_targets[1])

with open(saveFile, 'w') as f:
    json.dump(drug_targets, f, indent=2)

IndexError: list index out of range

In [13]:
# Önce JSON dosyasını bir DataFrame'e yükleyin
df = pd.read_json(drug_targets)

# Ardından DataFrame'i tekrar JSON dosyasına kaydedin
df.to_json('smaller_file.json', orient='records', lines=True, compression='gzip')

ValueError: Invalid file path or buffer object type: <class 'list'>

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import matplotlib.pyplot as plt

drug_toxicity = []
drug_food_interactions = []

for i in tqdm(range(len(drugs))):
    drug = drugs[i]
    idDB = drug[0].text  # Drug Bank ID

    
    for idx, feature in enumerate(drug):

        if 'toxicity' in str(feature):
            drug_toxicity_temp = drug[idx].text if drug[idx].text else None
            drug_toxicity.append(drug_toxicity_temp)

        if 'food-interactions' in str(feature):
            drug_food_interactions_temp = [fi.text for fi in list(drug[idx])] if list(drug[idx]) else None
            drug_food_interactions.append(drug_food_interactions_temp)

In [ ]:
import seaborn as sns
import pandas as pd


flat_list = [item for sublist in drug_food_interactions if sublist is not None for item in sublist]
all_text = ' '.join(flat_list)


# Kelimeleri ayırın ve küçük harfe çevirin
words = [word.lower() for word in word_tokenize(all_text) if word.isalpha()]

# Stopwords'leri kaldırın
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in words if word not in stop_words]

# Kelimeleri sayın
word_counts = Counter(filtered_words)

# En çok tekrar eden 100 kelimeyi bulun
top_words = word_counts.most_common(1000)

# Sonuçları görüntüleyin
print(top_words)